In [1]:
import os
import json
import tensorflow as tf

def load_tomato_disease_model(base_dir='models', model_name='TomatoDisease_acc_0.9871'):
    """
    Carga el modelo de detección de enfermedades en tomates guardado
    
    Args:
        base_dir: Directorio base donde está guardado el modelo
        model_name: Nombre base del modelo
        
    Returns:
        model: Modelo cargado
        model_info: Diccionario con información adicional del modelo
    """
    try:
        # 1. Cargar la configuración del modelo
        config_path = os.path.join(base_dir, f'{model_name}_config.json')
        with open(config_path, 'r') as f:
            config = json.load(f)
        
        # 2. Recrear el modelo desde la configuración
        model = tf.keras.models.model_from_config(eval(config))
        
        # 3. Cargar los pesos
        weights_path = os.path.join(base_dir, f'{model_name}_weights')
        model.load_weights(weights_path)
        
        # 4. Cargar información adicional
        info_path = os.path.join(base_dir, f'{model_name}_info.json')
        with open(info_path, 'r') as f:
            model_info = json.load(f)
            
        print("✓ Modelo cargado exitosamente")
        print(f"Precisión del modelo: {model_info['accuracy']:.4f}")
        print(f"Número de clases: {len(model_info['classes'])}")
        
        return model, model_info
        
    except Exception as e:
        print(f"Error al cargar el modelo: {str(e)}")
        return None, None

def predict_disease(model, image_path, model_info):
    """
    Realiza una predicción sobre una imagen
    
    Args:
        model: Modelo cargado
        image_path: Ruta a la imagen a analizar
        model_info: Información del modelo
        
    Returns:
        predicted_class: Clase predicha
        confidence: Confianza de la predicción
    """
    try:
        # Cargar y preprocesar la imagen
        img = tf.keras.preprocessing.image.load_img(
            image_path, 
            target_size=(model_info['image_size'][0], model_info['image_size'][1])
        )
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        
        # Realizar predicción
        predictions = model.predict(img_array)
        predicted_class_idx = tf.argmax(predictions[0]).numpy()
        
        # Obtener el nombre de la clase
        class_names = {v: k for k, v in model_info['classes'].items()}
        predicted_class = class_names[predicted_class_idx]
        confidence = float(predictions[0][predicted_class_idx])
        
        return predicted_class, confidence
        
    except Exception as e:
        print(f"Error al realizar la predicción: {str(e)}")
        return None, None